# Entrenamiento de una red neuronal para

### Obtención de datos

Inicialmente importamos la librería necesaria

In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
data = pd.read_csv("/home/anell/Projects/Pulsar-NN/pulsar_stars.csv")

In [3]:
data.head()

,Mean-of-the-integrated-profile,Standard-deviation-of the-integrated-profile,Excess-kurtosis-of-the-integrated-profile,Skewness-of-the-integrated-profile,Mean-of-the-DM-SNR-curve,Standard-deviation-of-the-DM-SNR-curve,Excess-kurtosis-of-the-DM-SNR-curve,Skewness-of-the-DM-SNR-curve,target_class
0,140.562500,55.683782,-0.234571,-0.699648,3.199833,19.110426,7.975532,74.242225,0
1,102.507812,58.882430,0.465318,-0.515088,1.677258,14.860146,10.576487,127.393580,0
2,103.015625,39.341649,0.323328,1.051164,3.121237,21.744669,7.735822,63.171909,0
3,136.750000,57.178449,-0.068415,-0.636238,3.642977,20.959280,6.896499,53.593661,0
4,88.726562,40.672225,0.600866,1.123492,1.178930,11.468720,14.269573,252.567306,0


In [4]:
ejemplos = data.values.tolist()
ejemplos = np.array(ejemplos)

features = ejemplos.transpose()

In [5]:
X = features[0:8]
Y = features[8]

X = X.transpose()

Ahora creamos el set de validación cruzada y el de entrenamiento

In [6]:
X_train = X[0:14000]
Y_train = Y[0:14000]

X_cv = X[14000:]
Y_cv = Y[14000:]

In [7]:
X_train.shape, Y_train.shape, X_cv.shape, X_cv.shape

((14000, 8), (14000,), (3898, 8), (3898, 8))

### Entrenamiento de la red neuronal

Importamos la libreria necesaria

In [8]:
from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout, Flatten
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


Primero armamos la red neuronal

In [9]:
inputs = Input(shape=(8,))

x = Dense(2)(inputs)
x = Activation("relu")(x)

x = Dense(1)(inputs)
x = Activation("sigmoid")(x)

model = Model(inputs, x)


# Compilación del modelo:
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 8)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
_________________________________________________________________
activation_2 (Activation)    (None, 1)                 0         
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________


Ahora entrenamos nuestra red neuronal

In [10]:
model.fit(X_train, Y_train, epochs=20, batch_size=10)  

Epoch 1/20
14000/14000 [==============================] - 23s 2ms/step - loss: 3.4791 - accuracy: 0.8272
Epoch 2/20
14000/14000 [==============================] - 18s 1ms/step - loss: 0.1398 - accuracy: 0.9633
Epoch 3/20
14000/14000 [==============================] - 16s 1ms/step - loss: 0.1000 - accuracy: 0.9706
Epoch 4/20
14000/14000 [==============================] - 16s 1ms/step - loss: 0.0995 - accuracy: 0.9712
Epoch 5/20
14000/14000 [==============================] - 19s 1ms/step - loss: 0.0987 - accuracy: 0.9721
Epoch 6/20
14000/14000 [==============================] - 18s 1ms/step - loss: 0.0981 - accuracy: 0.9709
Epoch 7/20
14000/14000 [==============================] - 15s 1ms/step - loss: 0.0985 - accuracy: 0.9720
Epoch 8/20
14000/14000 [==============================] - 19s 1ms/step - loss: 0.0981 - accuracy: 0.9723
Epoch 9/20
14000/14000 [==============================] - 19s 1ms/step - loss: 0.0979 - accuracy: 0.9723
Epoch 10/20
14000/14000 [==============================

Evaluamos el modelo para el conjunto de entrenamiento y el de validación cruzada:

In [11]:
# evaluación del modelo para el conjunto de entrenamiento
J_train, accuracy_train = model.evaluate(X_train, Y_train)
print('Accuracy train: %.2f' % (accuracy_train*100))

14000/14000 [==============================] - 2s 142us/step
Accuracy train: 96.91


In [12]:
# evaluación del modelo para el conjunto de validación cruzada
J_cv, accuracy_cv = model.evaluate(X_cv, Y_cv)
print('Accuracy CV: %.2f' % (accuracy_cv*100))

3898/3898 [==============================] - 0s 91us/step
Accuracy CV: 99.15
